# Data Wrangling Notebook
1. Load data
2. Evaluate
3. Tweak
5. Store

## Loading the data

In [132]:
import glob
import os
import pandas as pd
import numpy as np
import matplotlib
   
df = pd.concat(map(lambda file: pd.read_csv(file, sep=';'), glob.glob(os.path.join('', "./data/*.txt"))))

In [133]:
# Let's see what we got
# Remove blanks from column names first
df.columns = df.columns.str.strip()
df.sample(10)

,STATIONS_ID,MESS_DATUM,QN_3,FX,FM,QN_4,RSK,RSKF,SDK,SHK_TAG,NM,VPM,PM,TMK,UPM,TXK,TNK,TGK,eor
7118,430,19820628,-999,-999.0,-999.0,10,0.0,6,-999.0,0,6.3,13.6,1004.1,17.0,72.0,21.5,12.1,9.2,eor
5521,400,19760213,-999,-999.0,-999.0,5,0.0,0,0.0,-999,8.0,5.2,-999.0,0.8,81.0,2.1,-0.4,-1.5,eor
2404,407,19640801,-999,-999.0,-999.0,5,4.2,1,0.9,0,7.3,13.9,-999.0,13.9,87.0,20.5,12.8,12.4,eor
441,420,19940716,-999,-999.0,-999.0,10,0.0,0,-999.0,0,5.0,18.4,-999.0,21.3,72.0,27.3,18.2,17.0,eor
4105,432,19890627,-999,-999.0,-999.0,10,1.7,6,-999.0,0,6.7,17.3,-999.0,19.6,75.0,30.6,11.9,11.1,eor
21640,403,20090401,-999,-999.0,-999.0,10,0.0,0,11.7,0,1.1,7.4,1014.0,8.9,68.0,16.3,0.6,-2.7,eor
1917,2621,19610401,-999,-999.0,-999.0,5,-999.0,-999,-999.0,-999,-999.0,7.6,-999.0,4.5,91.0,8.3,-1.3,-4.2,eor
6102,417,19531015,-999,-999.0,-999.0,5,0.0,0,7.1,0,5.3,12.0,-999.0,13.2,81.0,19.5,5.2,-999.0,eor
1012,2621,19581009,-999,-999.0,-999.0,5,-999.0,-999,-999.0,-999,-999.0,12.8,-999.0,14.4,80.0,17.3,7.6,4.2,eor
17460,433,19951021,3,10.5,4.1,10,0.0,0,6.9,0,1.0,6.8,1020.6,5.4,74.0,10.5,3.7,2.3,eor


In [134]:
df.dtypes

STATIONS_ID      int64
MESS_DATUM       int64
QN_3             int64
FX             float64
FM             float64
QN_4             int64
RSK            float64
RSKF             int64
SDK            float64
SHK_TAG          int64
NM             float64
VPM            float64
PM             float64
TMK            float64
UPM            float64
TXK            float64
TNK            float64
TGK            float64
eor             object
dtype: object

| Column | Meaning | Unit |
| --- | --- | --- |
| STATIONS_ID | station id ||
| MESS_DATUM | date | yyyymmdd |
| QN_3 | quality level of next columns | coding see paragraph "Quality information" |
| FX | daily maximum of wind gust | m/s |
| FM | daily mean of wind speed | m/s |
| QN_4 | quality level of next columns | coding see paragraph "Quality information" |
| RSK | daily precipitation height | mm |
| RSKF | precipitation form ||
||no precipitation (conventional or automatic measurement), relates to WMO code 10 | 0 |
|| only rain (before 1979) | 1 |
||unknown form of recorded precipitation | 4 |
|| only rain; only liquid precipitation at automatic stations, relates to WMO code 11 | 6 |
|| only snow; only solid precipitation at automatic stations, relates to WMO code 12 | 7 |
|| rain and snow (and/or "Schneeregen"); liquid and solid precipitation at automatic stations, relates to WMO code 13 | 8 |
|| error or missing value or no automatic determination of precipitation form, relates to WMO code 15 | 9 |
| SDK | daily sunshine duration | h |
| SHK_TAG | daily snow depth | cm |
| NM | daily mean of cloud cover | 1/8 |
| VPM | daily mean of vapor pressure | hPa |
| PM | daily mean of pressure | hPa |
| TMK | daily mean of temperature | °C |
| UPM | daily mean of relative humidity | % |
| TXK | daily maximum of temperature at2m height | °C |
| TNK | daily minimum of temperature at 2m height | °C |
| TGK | daily minimum of air temperature at 5cm above ground | °C |
| eor | End of data record |

QN_3, QN_4 and RSKF are categorical values

$e^{i\pi} + 1 = 0$

In [135]:
mapping = {1: 'Winter', 2: 'Winter', 3: 'Spring', 4: 'Spring', 5: 'Spring', 6: 'Summer',
           7: 'Summer', 8: 'Summer', 9: 'Autumn', 10: 'Autumn', 11: 'Autumn', 12: 'Winter'}

In [136]:
(
    df
    .assign(MESS_DATUM = pd.to_datetime(df.MESS_DATUM, format='%Y%m%d'))
    .assign(MONTH=lambda df_: df_.MESS_DATUM.dt.month)
    .assign(SEASON=lambda df_: df_.MONTH.map(lambda x: mapping[x]))
    .assign(RSKF=lambda: df_: df_.RSKF.astype('category'))
    .drop('eor', axis=1)
    .drop('STATIONS_ID', axis=1)
    .replace(-999, np.nan)
    .sort_values(by='MESS_DATUM')
)

SyntaxError: invalid syntax (525729638.py, line 6)

In [ ]:
# Drop rows with more than 70 % nan values
clean = clean.drop(clean.columns[clean.isna().mean() > 0.7], axis=1)

In [ ]:
clean.MESS_DATUM[(clean.MESS_DATUM.dt.year > 1940) & (clean.MESS_DATUM.dt.year < 1970)].count()

In [ ]:
clean